<a href="https://colab.research.google.com/github/srinivasanibmbangalore/AnalyticsVidhyaCourseOnDL/blob/master/High_VariationIn_Data_Image_PreProcessing_Resizing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import argparse
import os
import shutil
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from psutil import virtual_memory
import tensorflow as tf

from keras import backend as K 
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation,InputLayer, Conv2D, MaxPool2D, Flatten
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.layers import GlobalMaxPool2D
import math
from keras.preprocessing.image import load_img, img_to_array,array_to_img
 

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = '/content/gdrive/My Drive/ml_data/avidhya/manvswoman'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
pwd='4/2gFMUCsxGjGljT8NT3AxVdk2byQziHV7GbeJ4ZXA7a05PEJPt8X6Gt0'

train_path=root_path+'/train'
test_path=root_path+'/test'
images_path=root_path+'/images'
tmp_images_path=images_path+'/581'
output_path=root_path+'/output'

In [4]:
!pwd


/content


In [5]:
import os
num_files=0
clmn_names=['image_name','imgDirLocation']
imgDF = pd.DataFrame(columns=clmn_names)
imgDF.describe()


,image_name,imgDirLocation
count,0,0
unique,0,0
top,NaN,NaN
freq,NaN,NaN


In [6]:
%%time
i=0
for root, dirs, files in tqdm(os.walk(tmp_images_path, topdown=False)):
   for name in files:
      _, ext = os.path.splitext(name)
      if (ext == '.jpg'):
          imgDF.loc[i]=[name,root]
          i+=1
    

1it [00:05,  5.94s/it]

CPU times: user 77.2 ms, sys: 1.02 ms, total: 78.2 ms
Wall time: 5.94 s


In [7]:
used_ramgb=virtual_memory().available/1e9
print("Available Memory is " + str(used_ramgb) + " gb")

Available Memory is 12.651216896 gb


In [10]:
imgDF.describe()

,image_name,imgDirLocation
count,23,23
unique,23,1
top,7707.jpg,/content/gdrive/My Drive/ml_data/avidhya/manvs...
freq,1,23


In [11]:
imgDF.head()

,image_name,imgDirLocation
0,13157.jpg,/content/gdrive/My Drive/ml_data/avidhya/manvs...
1,15526.jpg,/content/gdrive/My Drive/ml_data/avidhya/manvs...
2,2119.jpg,/content/gdrive/My Drive/ml_data/avidhya/manvs...
3,3561.jpg,/content/gdrive/My Drive/ml_data/avidhya/manvs...
4,17331.jpg,/content/gdrive/My Drive/ml_data/avidhya/manvs...


In [16]:
for index, row in imgDF.iterrows(): 
    imgLoc=row["imgDirLocation"]+'/'+row["image_name"]
    img=cv2.imread(imgLoc)
    print(imgLoc + " ... "+ str(img.shape))

/content/gdrive/My Drive/ml_data/avidhya/manvswoman/images/581/13157.jpg ... (224, 224, 3)
/content/gdrive/My Drive/ml_data/avidhya/manvswoman/images/581/15526.jpg ... (224, 224, 3)
/content/gdrive/My Drive/ml_data/avidhya/manvswoman/images/581/2119.jpg ... (224, 224, 3)
/content/gdrive/My Drive/ml_data/avidhya/manvswoman/images/581/3561.jpg ... (224, 224, 3)
/content/gdrive/My Drive/ml_data/avidhya/manvswoman/images/581/17331.jpg ... (224, 224, 3)
/content/gdrive/My Drive/ml_data/avidhya/manvswoman/images/581/17457.jpg ... (224, 224, 3)
/content/gdrive/My Drive/ml_data/avidhya/manvswoman/images/581/8352.jpg ... (224, 224, 3)
/content/gdrive/My Drive/ml_data/avidhya/manvswoman/images/581/5110.jpg ... (224, 224, 3)
/content/gdrive/My Drive/ml_data/avidhya/manvswoman/images/581/15240.jpg ... (224, 224, 3)
/content/gdrive/My Drive/ml_data/avidhya/manvswoman/images/581/10338.jpg ... (224, 224, 3)
/content/gdrive/My Drive/ml_data/avidhya/manvswoman/images/581/14638.jpg ... (224, 224, 3)
/co

In [26]:
## Resizing using OpenCV function
%%time
for index, row in imgDF.iterrows(): 
    imgLoc=row["imgDirLocation"]+'/'+row["image_name"]
    img=cv2.imread(imgLoc)
    resized_img=cv2.resize(img,(112,112))
    #print(imgLoc + " ... "+ str(resized_img.shape))



CPU times: user 31.1 ms, sys: 11.3 ms, total: 42.4 ms
Wall time: 108 ms


In [27]:
##  Load Image function of keras 
%%time

for index, row in imgDF.iterrows(): 
    imgLoc=row["imgDirLocation"]+'/'+row["image_name"]
    img=load_img(imgLoc,target_size=(112,112))
    imgArr=img_to_array(img)
    
### Observed that load time is far lower using Image Load Function of Keras

CPU times: user 36.1 ms, sys: 2.63 ms, total: 38.8 ms
Wall time: 67.5 ms


#### High Variation in Pixel Values data and reduction using scaling . 

In [29]:
%%time
i=0
for index, row in imgDF.iterrows(): 
    imgLoc=row["imgDirLocation"]+'/'+row["image_name"]
    img=load_img(imgLoc,target_size=(112,112))
    imgArr=img_to_array(img)
    if ( i ==4 ):
      print(imgArr)
      imgArr/=imgArr.max()
      print(imgArr)
    i+=1

[[[ 18.   9.  10.]
  [ 19.  10.  11.]
  [ 21.  11.  10.]
  ...
  [ 44.  26.  24.]
  [ 48.  30.  28.]
  [ 52.  34.  32.]]

 [[ 16.   7.   8.]
  [ 17.   8.   9.]
  [ 19.   9.   8.]
  ...
  [ 44.  26.  24.]
  [ 48.  30.  28.]
  [ 51.  33.  31.]]

 [[ 16.   7.   8.]
  [ 17.   8.   9.]
  [ 19.   9.   8.]
  ...
  [ 42.  24.  20.]
  [ 43.  25.  21.]
  [ 44.  26.  22.]]

 ...

 [[ 96.  89.  73.]
  [ 94.  87.  71.]
  [ 93.  86.  70.]
  ...
  [ 74. 174.  75.]
  [ 71. 179.  70.]
  [ 73. 183.  71.]]

 [[108.  94.  81.]
  [101.  89.  75.]
  [ 94.  82.  68.]
  ...
  [ 80. 174.  77.]
  [ 78. 176.  73.]
  [ 75. 178.  71.]]

 [[ 99.  83.  70.]
  [ 96.  82.  69.]
  [ 92.  78.  65.]
  ...
  [ 85. 175.  79.]
  [ 85. 182.  79.]
  [ 83. 182.  76.]]]
[[[0.075      0.0375     0.04166667]
  [0.07916667 0.04166667 0.04583333]
  [0.0875     0.04583333 0.04166667]
  ...
  [0.18333334 0.10833333 0.1       ]
  [0.2        0.125      0.11666667]
  [0.21666667 0.14166667 0.13333334]]

 [[0.06666667 0.02916667 0.03333